## Lab4
- in json file format, the string (rating) must be converted to integer for ALS (to recommend). 
- we use thge sha1 hashing algo to get the hex digit whichis 160 bits in length.It is cast to hexadecimal char, and then converted to int & then modulo-ed with 10^8 to get the 8-length representation.For the conversion:
    - we use withColumn, sha1 method the column intended, and then done by `hex_to_bin` which is a user-defined function implemented by us.
        - the func is done by lamda expression. we use pyspark.sql.functions.udf 
        - udf helps to speecify any custom logic in spark.
        - this function is to be done before using sha1
    - also modify the product id column , rating too.
- Then we use `ALS` for the recommendation. It is in the `pyspark.ml` module.
- Use random split to split into train and test data.
    - then als.fit on the training data.
    - to work on test data, als.transform
- We must evaluate the quality iof the model, we use the `pyspark.ml.evaluation` module.
 &rarr;

In [1]:
import findspark
findspark.init("C:/spark") 

from pyspark import SparkContext
sc= SparkContext("local","pyspark week4")

In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import *
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
df= spark.read.json("movies.json")


In [5]:
df.printSchema()
df.select('product_id',"user_id",'profile_name','review','score').show(5)

root
 |-- helpfulness: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- profile_name: string (nullable = true)
 |-- review: string (nullable = true)
 |-- score: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)

+----------+--------------+--------------------+--------------------+-----+
|product_id|       user_id|        profile_name|              review|score|
+----------+--------------+--------------------+--------------------+-----+
|B003AI2VGA|A141HP4LYPWMSR|Brian E. Erland "...|Synopsis: On the ...|  3.0|
|B003AI2VGA|A328S9RN3U5M68|          Grady Harp|THE VIRGIN OF JUA...|  3.0|
|B003AI2VGA|A1I7QGUDP043DG|Chrissy K. McVay ...|The scenes in thi...|  5.0|
|B003AI2VGA|A1M5405JH9THP9|        golgotha.gov|THE VIRGIN OF JUA...|  3.0|
|B003AI2VGA| ATXL536YX71TR|KerrLines "&#34;M...|Informationally, ...|  3.0|
+----------+--------------+--------------------+--------------------+-

In [5]:
hex_to_bigint= udf(lambda v: int(v,16)% (10**8) , LongType())

In [6]:
df= df.withColumn("user_hash", sha1(col("user_id")))
df= df.withColumn("product_hash", sha1(col("product_id")))

In [7]:
df.select('product_id',"user_id",'score','user_hash','product_hash').show(3)

+----------+--------------+-----+--------------------+--------------------+
|product_id|       user_id|score|           user_hash|        product_hash|
+----------+--------------+-----+--------------------+--------------------+
|B003AI2VGA|A141HP4LYPWMSR|  3.0|dcb0cc72376dc54f3...|3a1339ccd83d1fb05...|
|B003AI2VGA|A328S9RN3U5M68|  3.0|a20670dd0e673bd6a...|3a1339ccd83d1fb05...|
|B003AI2VGA|A1I7QGUDP043DG|  5.0|616addd183663c420...|3a1339ccd83d1fb05...|
+----------+--------------+-----+--------------------+--------------------+
only showing top 3 rows



In [8]:
df= df.withColumn("user_hasInt", hex_to_bigint(col("user_hash"))).withColumn("prod_hasInt", hex_to_bigint(col("product_hash")))

In [9]:
df.select('product_id',"user_id",'user_hash','user_hasInt','product_hash','prod_hasInt').show(3)


+----------+--------------+--------------------+-----------+--------------------+-----------+
|product_id|       user_id|           user_hash|user_hasInt|        product_hash|prod_hasInt|
+----------+--------------+--------------------+-----------+--------------------+-----------+
|B003AI2VGA|A141HP4LYPWMSR|dcb0cc72376dc54f3...|    5460385|3a1339ccd83d1fb05...|   51259877|
|B003AI2VGA|A328S9RN3U5M68|a20670dd0e673bd6a...|   64843361|3a1339ccd83d1fb05...|   51259877|
|B003AI2VGA|A1I7QGUDP043DG|616addd183663c420...|    1480848|3a1339ccd83d1fb05...|   51259877|
+----------+--------------+--------------------+-----------+--------------------+-----------+
only showing top 3 rows



In [10]:
(training,testing) = df.randomSplit([0.8,0.2])

In [12]:

als = ALS(
    userCol="user_hasInt",
    itemCol="prod_hasInt",
    ratingCol="score",
    rank=10,
    maxIter=10,
    regParam=0.1,
    coldStartStrategy="drop"
)

model = als.fit(training)


In [13]:
pred = model.transform(testing)
pred.select('product_id',"user_id",'score','prediction').show(5)


+----------+--------------+-----+----------+
|product_id|       user_id|score|prediction|
+----------+--------------+-----+----------+
|B00012FXJ0|A2B73CL3QSYWLB|  4.0|  2.100376|
|B003U6SJXQ|A2B73CL3QSYWLB|  3.0| 2.6146185|
|B0001NBNF2| A7QYHLIRZLMLQ|  2.0| 0.6517779|
|B000IMM3XW|A37DLVPKJZGSQF|  4.0|  4.054135|
|B004LB5FE0|A2B73CL3QSYWLB|  5.0|  4.691673|
+----------+--------------+-----+----------+
only showing top 5 rows



In [14]:

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="score",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(pred)
print(f"Root-mean-square error ={rmse} ")


Root-mean-square error =2.0694029591766503 


In [15]:
userRecs = model.recommendForAllUsers(5)   # top 5 items per user
userRecs.show(3,truncate=False)

+-----------+-------------------------------------------------------------------------------------------------------------------+
|user_hasInt|recommendations                                                                                                    |
+-----------+-------------------------------------------------------------------------------------------------------------------+
|5652       |[{80964704, 7.091551}, {79129662, 6.881047}, {17844814, 6.3622336}, {49252932, 6.2679725}, {53373833, 6.2468734}]  |
|12260      |[{81608461, 5.749242}, {78614700, 5.21922}, {19279346, 5.1858354}, {93086601, 5.051411}, {40695676, 5.041101}]     |
|19863      |[{19791041, 2.9653778}, {89474393, 2.2800643}, {34166803, 2.0979972}, {44425651, 1.9828594}, {18176630, 1.9585869}]|
+-----------+-------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



### hyperparam tuning 

In [11]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
als_tuned = ALS(
    userCol="user_hasInt",
    itemCol="prod_hasInt",
    ratingCol="score",
    rank=10,
    maxIter=10,
    regParam=0.1,
    coldStartStrategy="drop"
)

In [13]:
paramGrid= (ParamGridBuilder().addGrid(als_tuned.rank,[10,20]).addGrid(als_tuned.maxIter,[10,20]).\
        addGrid(als_tuned.regParam,[0.1,1.0]).build())

In [14]:
eval_tuned= RegressionEvaluator(
    metricName='rmse',
    labelCol= 'score',
    predictionCol= 'prediction'
)

In [15]:
cross= CrossValidator(
        estimator= als_tuned, estimatorParamMaps=paramGrid,
        evaluator=eval_tuned,   numFolds=3
)

cvModel= cross.fit(training)

In [ ]:
pred_tuned= cvModel.transform(testing)

In [21]:
rmse_tuned= eval_tuned.evaluate(pred_tuned)

print(rmse_tuned)

1.4874837385702886
